TRANSFORMAÇÃO LAND TO BRONZE

In [0]:
catalogo = "medalhao"
bronze_db_name = "bronze"

Ingestão das tabelas e adicionando campo de data de ingestão

In [0]:
from pyspark.sql import functions as F

def ingest_csv(nome_arquivo: str, nome_tabela: str):
    try:
        landing_path = f"/Volumes/{catalogo}/default/landing/{nome_arquivo}"
        tabela_completa = f"{catalogo}.{bronze_db_name}.{nome_tabela}"

        print(f"Iniciando ingestão de: {nome_arquivo} → Destino: {tabela_completa}")

        # Leitura do arquivo CSV
        df = (
            spark.read
                .option("header", "true")
                .option("inferSchema", "true")
                .csv(landing_path)
        )

        # Validação de arquivo vazio
        if df.count() == 0:
            raise ValueError(f"❌ O arquivo {nome_arquivo} está vazio ou não pôde ser lido.")

        # Adiciona timestamp de ingestão
        df_with_metadata = df.withColumn("ingestion_timestamp", F.current_timestamp())

        # Escrita no formato Delta
        (
            df_with_metadata
            .write
            .format("delta")
            .mode("overwrite") # Podia ser um append
            .saveAsTable(tabela_completa)
        )

        print(f"Tabela {tabela_completa} criada com sucesso!\n")

    except Exception as e:
        print(f"⚠️ Erro ao processar {nome_tabela}: {str(e)}\n")

In [0]:
ingest_csv("olist_customers_dataset.csv", "ft_consumidores")
ingest_csv("olist_geolocation_dataset.csv", "ft_geolocalizacao")
ingest_csv("olist_order_items_dataset.csv", "ft_itens_pedidos")
ingest_csv("olist_order_payments_dataset.csv", "ft_pagamentos_pedidos")
ingest_csv("olist_order_reviews_dataset.csv", "ft_avaliacoes_pedidos")
ingest_csv("olist_orders_dataset.csv", "ft_pedidos")
ingest_csv("olist_products_dataset.csv", "ft_produtos")
ingest_csv("olist_sellers_dataset.csv", "ft_vendedores")
ingest_csv("product_category_name_translation.csv", "dm_categoria_produtos_traducao")


Visualização das colunas e registros das tabelas

In [0]:
# Consumidores
df_consumidores = spark.table("medalhao.bronze.ft_consumidores")
display(df_consumidores.limit(10))
df_consumidores.printSchema()

In [0]:
# Geolocalização
df_geolocalizacao = spark.table("medalhao.bronze.ft_geolocalizacao")
display(df_geolocalizacao.limit(10))
df_geolocalizacao.printSchema()


In [0]:
# Itens de pedidos
df_itens_pedidos = spark.table("medalhao.bronze.ft_itens_pedidos")
display(df_itens_pedidos.limit(10))
df_itens_pedidos.printSchema()


In [0]:
# Pagamentos de pedidos
df_pagamentos_pedidos = spark.table("medalhao.bronze.ft_pagamentos_pedidos")
display(df_pagamentos_pedidos.limit(10))
df_pagamentos_pedidos.printSchema()

In [0]:
# Avaliações de pedidos
df_avaliacoes_pedidos = spark.table("medalhao.bronze.ft_avaliacoes_pedidos")
display(df_avaliacoes_pedidos.limit(10))
df_avaliacoes_pedidos.printSchema()

In [0]:
# Pedidos
df_pedidos = spark.table("medalhao.bronze.ft_pedidos")
display(df_pedidos.limit(10))
df_pedidos.printSchema()

In [0]:
# Produtos
df_produtos = spark.table("medalhao.bronze.ft_produtos")
display(df_produtos.limit(10))
df_produtos.printSchema()

In [0]:
# Categoria produtos tradução
df_categoria_traducao = spark.table("medalhao.bronze.dm_categoria_produtos_traducao")
display(df_categoria_traducao.limit(10))
df_categoria_traducao.printSchema()


In [0]:
df_categoria_traducao = spark.table("medalhao.bronze.dm_categoria_produtos_traducao")
display(df_categoria_traducao.limit(10))
df_categoria_traducao.printSchema()

Chamada da API

In [0]:
# Widgets no Databricks
dbutils.widgets.text("data_inicio_formatada", "01-01-2017", "Data início (MM-DD-AAAA)")
dbutils.widgets.text("data_fim_formatada", "12-31-2018", "Data fim (MM-DD-AAAA)")

# Lendo os valores dos widgets
data_inicio_formatada = dbutils.widgets.get("data_inicio_formatada")
data_fim_formatada = dbutils.widgets.get("data_fim_formatada")

print(f"Período selecionado: {data_inicio_formatada} até {data_fim_formatada}")


In [0]:
import requests

url = (
    "https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/"
    "CotacaoDolarPeriodo(dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)"
    f"?@dataInicial='{data_inicio_formatada}'"
    f"&@dataFinalCotacao='{data_fim_formatada}'"
    "&$select=dataHoraCotacao,cotacaoCompra"
    "&$format=json"
)

print(f"🔗 Requisitando dados da API do Banco Central...\n{url}")

# Requisição HTTP
response = requests.get(url)
response.raise_for_status()

# Extrai o conteúdo JSON
dados_json = response.json().get("value", [])

if not dados_json:
    raise ValueError(" Nenhum dado retornado pela API. Verifique o intervalo de datas.")
else:
    print(f" {len(dados_json)} registros obtidos da API PTAX.")


In [0]:
from pyspark.sql import functions as F

df_cotacao = (
    spark.createDataFrame(dados_json)
        .withColumn("ingestion_timestamp", F.current_timestamp()) 
)

display(df_cotacao.limit(5))
df_cotacao.printSchema()


In [0]:
(
    df_cotacao.write
        .format("delta")
        .mode("overwrite")  # substitui se já existir
        .saveAsTable(f"{catalogo}.{bronze_db_name}.dm_cotacao_dolar")
)

print("Tabela bronze.dm_cotacao_dolar criada com sucesso!")

In [0]:
df_categoria_traducao = spark.table("medalhao.bronze.dm_cotacao_dolar")
display(df_categoria_traducao.limit(10))
df_categoria_traducao.printSchema()